In [1]:
import pandas as pd
from scipy import stats
import seaborn as sns
import numpy as np


In [2]:
df = pd.read_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/A1_카드/Telemarketing_Data.csv')

In [3]:
df.head()

,연령,직업군,결혼여부,학력,신용카드소지여부,주택대출여부,개인대출여부,contact,통화시간,연락시도횟수,p_days,과거통화횟수,계약여부,카드종류,연락일
0,56,주부,결혼,중졸,없음,없음,없음,유선전화,261,1,NaN,0,no,87,2020-01-03
1,57,서비스종사자,결혼,전문학교,미확인,없음,없음,유선전화,149,1,NaN,0,no,88,2020-01-03
2,37,서비스종사자,결혼,전문학교,없음,대출있음,없음,유선전화,226,1,NaN,0,no,89,2020-01-03
3,40,회사원,결혼,검정고시,없음,없음,없음,유선전화,151,1,NaN,0,no,90,2020-01-03
4,56,서비스종사자,결혼,전문학교,없음,없음,대출있음,유선전화,307,1,NaN,0,no,91,2020-01-03


In [4]:
df.isna().sum()

연령              0
직업군             0
결혼여부            0
학력              0
신용카드소지여부        0
주택대출여부          0
개인대출여부          0
contact         0
통화시간            0
연락시도횟수          0
p_days      39673
과거통화횟수          0
계약여부            0
카드종류            0
연락일             0
dtype: int64

In [5]:
# pdays는 Moro(2014)의 논문 내용대로 -1로 처리 
df['p_days'] = df['p_days'].fillna(-1)

연령대

In [6]:
def age_group(x):
    if x<10:
        return '유아'
    elif 10<=x<20:
        return '10대'
    elif 20<=x<30:
        return '20대'
    elif 30<=x<40:
        return '30대'
    elif 40<=x<50:
        return '40대'
    elif 50<=x<60:
        return '50대'
    elif 60<=x<70:
        return '60대'
    elif 70<=x<80:
        return '70대'
    elif 80<=x<90:
        return '80대'
    else:
        return '90대'

In [7]:
df['연령대'] = df['연령'].apply(age_group)

In [8]:
df.head()

,연령,직업군,결혼여부,학력,신용카드소지여부,주택대출여부,개인대출여부,contact,통화시간,연락시도횟수,p_days,과거통화횟수,계약여부,카드종류,연락일,연령대
0,56,주부,결혼,중졸,없음,없음,없음,유선전화,261,1,-1.0,0,no,87,2020-01-03,50대
1,57,서비스종사자,결혼,전문학교,미확인,없음,없음,유선전화,149,1,-1.0,0,no,88,2020-01-03,50대
2,37,서비스종사자,결혼,전문학교,없음,대출있음,없음,유선전화,226,1,-1.0,0,no,89,2020-01-03,30대
3,40,회사원,결혼,검정고시,없음,없음,없음,유선전화,151,1,-1.0,0,no,90,2020-01-03,40대
4,56,서비스종사자,결혼,전문학교,없음,없음,대출있음,유선전화,307,1,-1.0,0,no,91,2020-01-03,50대


# 결측치 채우기 
-> 미확인 결측치 채우기: 각 명목변수들의 카이제곱 검정을 통해 어떤 col이 상관성을 가장 많이 가지는 지를 기반으로,  
이 col의 기준으로 각 col당(결혼여부, 학력, 시뇽카드 소지여부, 주택 대출여부, 개인대출여부)를 나눠 테이블을 형성해  
해당 직업의 다른 col의 미확인이 있을시 그 col을 위의 테이블로 채움   
ex 직업군 주부의 학력이 전문학교가 많다면
df의 직업군 주부의 학력 '미확인'값은 '전문학교'로 대체 

각 col의 상관성 판단

pivot 테이블 형성 후 이를 기반으로 카이제곱 검정을 해서 상관성 검정

In [9]:
# idx로 만들 df의 col명, col으로 만들 df의 col명 지정
# ex make_pivot(df, '직업군', '연령대') return 직원군별 연령대 명수 카운트
def make_pivot(df,idx, col):
    tmp = pd.DataFrame(index = df[idx].unique(), columns = df[col].unique())
    for i in df[idx].unique():
        for j in df[col].unique():
            tmp.loc[i,j] = len(df[(df[idx] == i) & (df[col]== j)])
    if '미확인' in tmp.columns:
        tmp.drop('미확인', axis = 1,inplace = True)
    if '미확인' in tmp.T.columns:
        tmp.drop('미확인', axis = 0, inplace = True)
    return tmp

In [10]:
make_pivot(df, '연령대','직업군')

,주부,서비스종사자,회사원,산업노동자,기술자,은퇴,공무원,미취업자,자영업자,기업가,학생
50대,323,514,1325,1416,906,826,703,170,268,319,0
30대,313,1694,4894,3777,3364,27,1072,415,566,558,185
40대,312,1032,2475,2854,1622,78,910,284,381,459,8
20대,45,721,1622,1164,802,4,181,135,185,103,607
60대,42,8,98,40,48,366,52,10,20,17,0
10대,0,0,0,0,0,0,0,0,0,0,75
70대,12,0,8,2,1,286,5,0,1,0,0
80대,13,0,0,1,0,123,1,0,0,0,0
90대,0,0,0,0,0,10,0,0,0,0,0


In [11]:
# df_raw
def col_corr_chk(df):
    for i in ['연령대','직업군','결혼여부','학력','신용카드소지여부','주택대출여부','개인대출여부']:
        corr= [];
        for j in ['연령대','직업군','결혼여부','학력','신용카드소지여부','주택대출여부','개인대출여부']:
            chi, pval, dof, expected = stats.chi2_contingency(make_pivot(df, i,j))
            if pval < 0.05:
                # print(f'{'직업군'}과{'학력'}은 연관성이 있다.)
                corr.append(j)
        print(f'{i}과 {corr}은 연관성이있다.') #  직업군으로 나누는 것이 합리적임
        

In [12]:
col_corr_chk(df)

연령대과 ['연령대', '직업군', '결혼여부', '학력', '주택대출여부']은 연관성이있다.
직업군과 ['연령대', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부']은 연관성이있다.
결혼여부과 ['연령대', '직업군', '결혼여부', '학력', '주택대출여부']은 연관성이있다.
학력과 ['연령대', '직업군', '결혼여부', '학력', '주택대출여부']은 연관성이있다.
신용카드소지여부과 ['직업군', '신용카드소지여부']은 연관성이있다.
주택대출여부과 ['연령대', '직업군', '결혼여부', '학력', '주택대출여부', '개인대출여부']은 연관성이있다.
개인대출여부과 ['직업군', '주택대출여부', '개인대출여부']은 연관성이있다.


-> 직업군으로 결측치 처리 대상 선정

# 각 직업군별로 명목항목값의 가장 많은 걸로 테이블 형성


In [13]:
# tmp = make_pivot(df, '직업군','학력')
tmp = make_pivot(df, '직업군','결혼여부')
tmp

,결혼,미혼,이혼
주부,777,119,161
서비스종사자,2294,1137,532
회사원,5253,3875,1280
산업노동자,6687,1825,728
기술자,3670,2287,774
은퇴,1274,93,348
공무원,2089,501,331
미취업자,634,251,124
자영업자,904,379,133
기업가,1071,203,179


['연령대', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부']

In [14]:
col = '연령대'
tmp = make_pivot(df, '직업군',col)
total_job = []; total_info = [];
for job in tmp.T.columns:
    # 각 직업군당 max값을 추출해 max값에 해당하는 col추출 
    col_ext = tmp.columns

    each_row_max_bool = tmp.loc[job] == max(tmp.loc[job]) # max값에 해당하는 col(ex 학력)을 [T,F,F,F,] 이런식으로 추출해서 각 직업군에 유력 학력을 형성
    for idx, bool in zip(range(len(each_row_max_bool)),
                        each_row_max_bool):
        if bool == True:
            # print(job, col_ext[idx])
            total_job.append(job); total_info.append(col_ext[idx])


In [15]:
make_pivot(df, '직업군','연령대')

,50대,30대,40대,20대,60대,10대,70대,80대,90대
주부,323,313,312,45,42,0,12,13,0
서비스종사자,514,1694,1032,721,8,0,0,0,0
회사원,1325,4894,2475,1622,98,0,8,0,0
산업노동자,1416,3777,2854,1164,40,0,2,1,0
기술자,906,3364,1622,802,48,0,1,0,0
은퇴,826,27,78,4,366,0,286,123,10
공무원,703,1072,910,181,52,0,5,1,0
미취업자,170,415,284,135,10,0,0,0,0
자영업자,268,566,381,185,20,0,1,0,0
기업가,319,558,459,103,17,0,0,0,0


# 직업군의 미확인값 채우기 
연령대 이용해서 각 연령대별 가장 많은 걸 넣음

In [16]:
make_pivot(df, '연령대','직업군')

,주부,서비스종사자,회사원,산업노동자,기술자,은퇴,공무원,미취업자,자영업자,기업가,학생
50대,323,514,1325,1416,906,826,703,170,268,319,0
30대,313,1694,4894,3777,3364,27,1072,415,566,558,185
40대,312,1032,2475,2854,1622,78,910,284,381,459,8
20대,45,721,1622,1164,802,4,181,135,185,103,607
60대,42,8,98,40,48,366,52,10,20,17,0
10대,0,0,0,0,0,0,0,0,0,0,75
70대,12,0,8,2,1,286,5,0,1,0,0
80대,13,0,0,1,0,123,1,0,0,0,0
90대,0,0,0,0,0,10,0,0,0,0,0


## 직업군별 가장 많은 값들 추산

In [17]:
def job_missing_fill(df):
    col = '직업군'
    tmp =make_pivot(df, '연령대','직업군')
    total_job = []; total_info = [];
    for job in tmp.T.columns:
        # 각 직업군당 max값을 추출해 max값에 해당하는 col추출 
        col_ext = tmp.columns

        each_row_max_bool = tmp.loc[job] == max(tmp.loc[job]) # max값에 해당하는 col(ex 학력)을 [T,F,F,F,] 이런식으로 추출해서 각 직업군에 유력 학력을 형성
        for idx, bool in zip(range(len(list(each_row_max_bool))),
                            list(each_row_max_bool)):
            if bool == True:
                # print(job, col_ext[idx])
                total_job.append(job); total_info.append(col_ext[idx])


        total_gb_job = pd.DataFrame(data = {'연령대':total_job,
                        col:total_info})
        total_gb_job = total_gb_job.set_index('연령대')
    return total_gb_job

In [18]:
job_fill = job_missing_fill(df)

In [19]:
job_fill

,직업군
연령대,
50대,산업노동자
30대,회사원
40대,산업노동자
20대,회사원
60대,은퇴
10대,학생
70대,은퇴
80대,은퇴
90대,은퇴


In [20]:
def fill_job(df):
    # df 직업군 미확인값 채우기
    for age in job_fill.index:
        df.loc[(df['연령대'] == age) & (df['직업군'] == '미확인'),'직업군'] = job_fill.loc[age,'직업군']
    return df

In [21]:
df = fill_job(df)
dict_ = dict.fromkeys(df.columns)
for i in df.columns:
    dict_[i] = len(df[df[i]=='미확인'])

In [22]:
dict_ # 실제로 직업군 결측치 처리 완료

{'연령': 0,
 '직업군': 0,
 '결혼여부': 80,
 '학력': 1731,
 '신용카드소지여부': 8597,
 '주택대출여부': 990,
 '개인대출여부': 990,
 'contact': 0,
 '통화시간': 0,
 '연락시도횟수': 0,
 'p_days': 0,
 '과거통화횟수': 0,
 '계약여부': 0,
 '카드종류': 0,
 '연락일': 0,
 '연령대': 0}

각 직업별로 체크해서 값 넣기

In [23]:
def total_gb_job(df):
    for col in ['결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부']:
        tmp = make_pivot(df, '직업군',col)
        total_job = []; total_info = [];
        for job in tmp.T.columns:
            # 각 직업군당 max값을 추출해 max값에 해당하는 col추출 
            col_ext = list(tmp.columns)

            each_row_max_bool = tmp.loc[job] == max(tmp.loc[job]) # max값에 해당하는 col(ex 학력)을 [T,F,F,F,] 이런식으로 추출해서 각 직업군에 유력 학력을 형성
            for idx, bool in zip(range(len(list(each_row_max_bool))),
                                list(each_row_max_bool)):
                if bool == True:
                    # print(job, col_ext[idx])
                    total_job.append(job); total_info.append(col_ext[idx])

        if col == '결혼여부':
            total_gb_job = pd.DataFrame(data = {'직업군':total_job,
                            col:total_info})

        else:
            gb_job = pd.DataFrame(data = {'직업군':total_job,
                            col:total_info})
            total_gb_job = total_gb_job.merge(gb_job, on = '직업군')
    
    total_gb_job = total_gb_job.set_index('직업군')
    return total_gb_job


In [24]:
total_gb_job = total_gb_job(df)

In [25]:
total_gb_job

,결혼여부,학력,신용카드소지여부,주택대출여부,개인대출여부
직업군,,,,,
주부,결혼,중졸,없음,대출있음,없음
서비스종사자,결혼,전문학교,없음,대출있음,없음
회사원,결혼,대졸학사,없음,대출있음,없음
산업노동자,결혼,고졸,없음,대출있음,없음
기술자,결혼,석박사,없음,대출있음,없음
은퇴,결혼,중졸,없음,대출있음,없음
공무원,결혼,대졸학사,없음,대출있음,없음
미취업자,결혼,대졸학사,없음,대출있음,없음
자영업자,결혼,대졸학사,없음,대출있음,없음


In [26]:
df[(df['직업군'] == job) & (df['결혼여부'] == '미확인')][['직업군','결혼여부']]

,직업군,결혼여부
34423,학생,미확인


In [27]:
# 결측치 처리가 된 직업군을 기준으로 나머지 결혼여부~ 개인대출 여부 col의 미확인 값 처리 
def fill_all_ext(df):
    for fill_col in total_gb_job.columns:
        for job in total_gb_job.index:
            df.loc[(df['직업군'] == job) & (df[fill_col] == '미확인'),fill_col] = total_gb_job.loc[job,fill_col]
    return df

In [28]:
'''for job, ch_col in zip(total_gb_job.index,total_gb_job.columns):
    df.loc[(df['직업군'] == job) & (df[ch_col] == '미확인'),ch_col] = total_gb_job.loc[job,ch_col]'''

"for job, ch_col in zip(total_gb_job.index,total_gb_job.columns):\n    df.loc[(df['직업군'] == job) & (df[ch_col] == '미확인'),ch_col] = total_gb_job.loc[job,ch_col]"

In [29]:
'''ch_col = '결혼여부'
for ch_col in total_gb_job.columns:
    df.loc[(df['직업군'] == job) & (df[ch_col] == '미확인'), ch_col] = total_gb_job.loc[job, ch_col]'''

"ch_col = '결혼여부'\nfor ch_col in total_gb_job.columns:\n    df.loc[(df['직업군'] == job) & (df[ch_col] == '미확인'), ch_col] = total_gb_job.loc[job, ch_col]"

In [30]:
df = fill_all_ext(df)
dict_ = dict.fromkeys(df.columns)
for i in df.columns:
    dict_[i] = len(df[df[i]=='미확인'])

In [31]:
print(dict_) # 미확인값도 다 처리됨
print(df.isna().sum()) # 결측치처리도 완료

{'연령': 0, '직업군': 0, '결혼여부': 0, '학력': 0, '신용카드소지여부': 0, '주택대출여부': 0, '개인대출여부': 0, 'contact': 0, '통화시간': 0, '연락시도횟수': 0, 'p_days': 0, '과거통화횟수': 0, '계약여부': 0, '카드종류': 0, '연락일': 0, '연령대': 0}
연령          0
직업군         0
결혼여부        0
학력          0
신용카드소지여부    0
주택대출여부      0
개인대출여부      0
contact     0
통화시간        0
연락시도횟수      0
p_days      0
과거통화횟수      0
계약여부        0
카드종류        0
연락일         0
연령대         0
dtype: int64


---


# 파생변수 생성


1. 연령대

In [32]:
def age_group(x):
    if x<10:
        return '유아'
    elif 10<=x<20:
        return '10대'
    elif 20<=x<30:
        return '20대'
    elif 30<=x<40:
        return '30대'
    elif 40<=x<50:
        return '40대'
    elif 50<=x<60:
        return '50대'
    elif 60<=x<70:
        return '60대'
    elif 70<=x<80:
        return '70대'
    elif 80<=x<90:
        return '80대'
    else:
        return '90대'

2. 년도, 요일, 달, 일, 공휴일

In [33]:
def ch_day(x):
    if x == 'Monday':
        return '월요일'
    elif x == 'Tuesday':
        return '화요일'
    elif x == 'Wednesday':
        return '수요일'
    elif x == 'Thursday':
        return '목요일'
    elif x == 'Friday':
        return '금요일'
    elif x == 'Saturday':
        return '토요일'
    else:
        return '일요일'
    

In [34]:
df.columns

Index(['연령', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류', '연락일', '연령대'],
      dtype='object')

In [35]:
df['연락일'] = pd.to_datetime(df['연락일'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 16 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   연령        41188 non-null  int64         
 1   직업군       41188 non-null  object        
 2   결혼여부      41188 non-null  object        
 3   학력        41188 non-null  object        
 4   신용카드소지여부  41188 non-null  object        
 5   주택대출여부    41188 non-null  object        
 6   개인대출여부    41188 non-null  object        
 7   contact   41188 non-null  object        
 8   통화시간      41188 non-null  int64         
 9   연락시도횟수    41188 non-null  int64         
 10  p_days    41188 non-null  float64       
 11  과거통화횟수    41188 non-null  int64         
 12  계약여부      41188 non-null  object        
 13  카드종류      41188 non-null  int64         
 14  연락일       41188 non-null  datetime64[ns]
 15  연령대       41188 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(5), object(9)
memo

In [36]:
df['연락년도'] = df['연락일'].dt.year # dt.year 함수 사용
df['연락달'] = df['연락일'].dt.month #월
df['연락날'] = df['연락일'].dt.day # 몇일에 
df['연락요일'] = df['연락일'].dt.day_name() # 몇요일인지

In [37]:
df.head()

,연령,직업군,결혼여부,학력,신용카드소지여부,주택대출여부,개인대출여부,contact,통화시간,연락시도횟수,p_days,과거통화횟수,계약여부,카드종류,연락일,연령대,연락년도,연락달,연락날,연락요일
0,56,주부,결혼,중졸,없음,없음,없음,유선전화,261,1,-1.0,0,no,87,2020-01-03,50대,2020,1,3,Friday
1,57,서비스종사자,결혼,전문학교,없음,없음,없음,유선전화,149,1,-1.0,0,no,88,2020-01-03,50대,2020,1,3,Friday
2,37,서비스종사자,결혼,전문학교,없음,대출있음,없음,유선전화,226,1,-1.0,0,no,89,2020-01-03,30대,2020,1,3,Friday
3,40,회사원,결혼,검정고시,없음,없음,없음,유선전화,151,1,-1.0,0,no,90,2020-01-03,40대,2020,1,3,Friday
4,56,서비스종사자,결혼,전문학교,없음,없음,대출있음,유선전화,307,1,-1.0,0,no,91,2020-01-03,50대,2020,1,3,Friday


In [38]:
df['연락요일'] = df['연락요일'].apply(ch_day)

## 공휴일과 평일의 텔레마케팅 성공여부

In [39]:
from pytimekr import pytimekr

list_2020 = pytimekr.holidays(2020) # 2020  holidays메소드는 리스트 형태로 관련값 반환
list_2021 = pytimekr.holidays(2021) # 2021 holidays메소드는 리스트 형태로 관련값 반환
list = list_2020 + list_2021


#### 공휴일과 평일 함수

In [40]:
def day_week(x):
    list_2020 = pytimekr.holidays(2020) # 2020  holidays메소드는 리스트 형태로 관련값 반환
    list_2021 = pytimekr.holidays(2021) # 2021 holidays메소드는 리스트 형태로 관련값 반환
    list = list_2020 + list_2021
    if x.date() in list:
        return '공휴일'
    else:
        return '평일'

In [41]:
df['공휴일과평일'] = df['연락일'].apply(day_week)

#### 공휴일과 평일의 계약여부는 차이가 있을까?

In [42]:
df.groupby(['공휴일과평일','계약여부'])['계약여부'].agg('count')

공휴일과평일  계약여부
공휴일     no       1538
        yes       170
평일      no      35010
        yes      4470
Name: 계약여부, dtype: int64

#### 요일별로 차이가 있을까?

In [43]:
df.groupby(['연락요일','계약여부'])['계약여부'].agg('count')

연락요일  계약여부
금요일   no      4849
      yes      659
목요일   no      4694
      yes      640
수요일   no      4735
      yes      526
월요일   no      5926
      yes      737
일요일   no      5250
      yes      648
토요일   no      4984
      yes      658
화요일   no      6110
      yes      772
Name: 계약여부, dtype: int64

## 카드종류별 혜택에 따른 텔레마케팅 가입여부는 차이가 있을까?

In [44]:
# 카드 종류별 혜택사항
card = pd.read_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/A1_카드/Card_Data.csv')

df = df.merge( card[['Card_ID','혜택분야','연회비']], left_on ='카드종류', right_on = 'Card_ID',how='left') # 혜택분야, 연회비 둘다 가져옴-> 연회비 이후 사용하기 위해서 
df.drop('Card_ID',axis = 1,inplace = True) # 오른쪽 df의 키로 사용한 CardID 제거

## 연회비 기준 
- 없음/ 최초1회 납부/ 지속 납부
> 연회비를 1회 납부하거나 지속 납부할 경우 카드사의 충성도가 높으므로, 텔레마케팅 아웃바운드 성공률이 높을 것이다. 


In [45]:
def split(x):
    if x == '없음' or x ==  '면제': # 아예 내지 않는 경우 
        return 0
    elif '최초' in x: # 한번만 내는 경우
        return 1
    else:
        return 2 # 지속 납부의 경우

In [46]:
df['연회비납부정도'] = df['연회비'].apply(split)
df.head()

,연령,직업군,결혼여부,학력,신용카드소지여부,주택대출여부,개인대출여부,contact,통화시간,연락시도횟수,...,연락일,연령대,연락년도,연락달,연락날,연락요일,공휴일과평일,혜택분야,연회비,연회비납부정도
0,56,주부,결혼,중졸,없음,없음,없음,유선전화,261,1,...,2020-01-03,50대,2020,1,3,금요일,평일,life,Visa 1만 8천원\nLocal(국내전용) 1만 8천원,2
1,57,서비스종사자,결혼,전문학교,없음,없음,없음,유선전화,149,1,...,2020-01-03,50대,2020,1,3,금요일,평일,life,VISA 3만원\nURS 3만원,2
2,37,서비스종사자,결혼,전문학교,없음,대출있음,없음,유선전화,226,1,...,2020-01-03,30대,2020,1,3,금요일,평일,life,VISA 2만원\nURS 2만원,2
3,40,회사원,결혼,검정고시,없음,없음,없음,유선전화,151,1,...,2020-01-03,40대,2020,1,3,금요일,평일,life,VISA 3만 3천원\nUPI 3만원,2
4,56,서비스종사자,결혼,전문학교,없음,없음,대출있음,유선전화,307,1,...,2020-01-03,50대,2020,1,3,금요일,평일,life,VISA 3만 3천원\nUPI 3만원,2


# 학벌 피쳐 나누기

In [47]:
def scholar(x):
    if x == '전문학교' or x == '석박사' or x == '대졸학사':
        return '대학교이상'
    else:
        return '대학교미만'

In [48]:
df['대학교구분'] = df.학력.apply(scholar)
df.head()

,연령,직업군,결혼여부,학력,신용카드소지여부,주택대출여부,개인대출여부,contact,통화시간,연락시도횟수,...,연령대,연락년도,연락달,연락날,연락요일,공휴일과평일,혜택분야,연회비,연회비납부정도,대학교구분
0,56,주부,결혼,중졸,없음,없음,없음,유선전화,261,1,...,50대,2020,1,3,금요일,평일,life,Visa 1만 8천원\nLocal(국내전용) 1만 8천원,2,대학교미만
1,57,서비스종사자,결혼,전문학교,없음,없음,없음,유선전화,149,1,...,50대,2020,1,3,금요일,평일,life,VISA 3만원\nURS 3만원,2,대학교이상
2,37,서비스종사자,결혼,전문학교,없음,대출있음,없음,유선전화,226,1,...,30대,2020,1,3,금요일,평일,life,VISA 2만원\nURS 2만원,2,대학교이상
3,40,회사원,결혼,검정고시,없음,없음,없음,유선전화,151,1,...,40대,2020,1,3,금요일,평일,life,VISA 3만 3천원\nUPI 3만원,2,대학교미만
4,56,서비스종사자,결혼,전문학교,없음,없음,대출있음,유선전화,307,1,...,50대,2020,1,3,금요일,평일,life,VISA 3만 3천원\nUPI 3만원,2,대학교이상


# 통화시간 초-> 분

In [49]:
df['통화시간_분'] = df['통화시간']/60

In [50]:
df.columns

Index(['연령', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류', '연락일', '연령대',
       '연락년도', '연락달', '연락날', '연락요일', '공휴일과평일', '혜택분야', '연회비', '연회비납부정도',
       '대학교구분', '통화시간_분'],
      dtype='object')

# EDA 이후 파생변수 재그룹화
1. 연령대

In [51]:
# 20대이하/30대/40대/50대/60대이상으로 나눔
def age_group(x):
    if x<30:
        return '20대 이하'
    elif 30<=x<40:
        return '30대'
    elif 40<=x<50:
        return '40대'
    elif 50<=x<60:
        return '50대'
    else :
        return '60대이상'
    
df['연령대'] = df['연령'].apply(age_group)

2. 결혼여부

In [52]:
# 미혼+이혼수를 합치면 결혼수와 수가 비슷해지니까 합쳐본다.
# 미혼+이혼을 합침
def marry_group(x):
    if x == '미혼' or x == '이혼' :
        return '미혼'
    else :
        return '기혼'

df['결혼여부'] = df.결혼여부.apply(marry_group)

3. 학력

In [53]:
##대학교 구분별 - 대졸학사이상/ 대졸미만     
def scholar(x) :
        if  x == '석박사' or x ==  '대졸학사':
            return '4년제 대학교이상'
        else :
            return '대학교미만'
        
df["대학교구분"]= df.학력.apply(scholar)  

4. 직업군-> 직업급여별

In [54]:
def collar3(x) :
        if x == '산업노동자' :
            return '산업노동자'
        
        elif x == '회사원' or x == '공무원' or x == '서비스종사자':
             return '월급쟁이'
        elif  x == '주부' or x == '은퇴' or  x == '미취업자' or x == '학생' :
            return '무급'
        
        elif x == '기술자'or x =='자영업자' or x == '기업가':
            return '사업가'
        
        
df["직업급여별"] = df.직업군.apply(collar3)    

## 계약 비율

In [55]:
#연령대별 계약여부
def prop(df,col):

    tmp = df.groupby([col], as_index = False)['계약여부'].value_counts().rename(columns = {'count':'총합'})
    count =tmp.groupby([col], as_index = False)['총합'].agg('sum').rename(columns = {'총합':col +'별총합'})
    tmp = tmp.merge(count,on = col, how = 'left')
    tmp['비율'] = tmp['총합']/tmp[col +'별총합']
    return tmp


In [56]:
tmp

,50대,30대,40대,20대,60대,10대,70대,80대,90대
주부,323,313,312,45,42,0,12,13,0
서비스종사자,514,1694,1032,721,8,0,0,0,0
회사원,1325,4894,2475,1622,98,0,8,0,0
산업노동자,1416,3777,2854,1164,40,0,2,1,0
기술자,906,3364,1622,802,48,0,1,0,0
은퇴,826,27,78,4,366,0,286,123,10
공무원,703,1072,910,181,52,0,5,1,0
미취업자,170,415,284,135,10,0,0,0,0
자영업자,268,566,381,185,20,0,1,0,0
기업가,319,558,459,103,17,0,0,0,0


# 모델링 df 만들기
- 그대로 가져다 하면 accuracy가 너무 낮고 몇개 제거하면 accuracy가 너무 좋음
카테고리컬 변수중에 제거할 거 한 뒤 돌리기

In [57]:
df.columns

Index(['연령', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류', '연락일', '연령대',
       '연락년도', '연락달', '연락날', '연락요일', '공휴일과평일', '혜택분야', '연회비', '연회비납부정도',
       '대학교구분', '통화시간_분', '직업급여별'],
      dtype='object')

In [58]:
def age_group(x):
    if x<36:
        return 'MZ세대'
    elif 40<=x<49:
        return '영포티'
    elif 50<=x<59:
        return 'Old-X세대'
    elif 59<x<=73:
        return '베이비붐'
    else:
        return '산업화'


In [59]:
df['연령대_재범주화'] = df['연령'].apply(age_group)

In [60]:
df.columns

Index(['연령', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류', '연락일', '연령대',
       '연락년도', '연락달', '연락날', '연락요일', '공휴일과평일', '혜택분야', '연회비', '연회비납부정도',
       '대학교구분', '통화시간_분', '직업급여별', '연령대_재범주화'],
      dtype='object')

In [65]:
df['통화시간(분)'] = df['통화시간']/60
df['통화시간(분)'] = df['통화시간(분)'].astype(int)

In [69]:
df  = df.drop(['연령','직업군','학력','혜택분야','p_days','연락일','연락년도','연락날','공휴일과평일','통화시간','통화시간_분'], axis = 1)

In [70]:
df.columns

Index(['결혼여부', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact', '연락시도횟수', '과거통화횟수',
       '계약여부', '카드종류', '연령대', '연락달', '연락요일', '연회비', '연회비납부정도', '대학교구분',
       '직업급여별', '연령대_재범주화', '통화시간(분)'],
      dtype='object')

In [71]:
temp = df.copy()


In [72]:
temp.to_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/ipynb파일/모델링_csv파일/model_chk.csv')

In [ ]:
temp.to_csv('텔레마케팅_모델링.csv')

In [ ]:
df.to_csv('텔레마케팅_파생변수.csv')

In [ ]:
# temp2 = df[['연령', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
#        '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '연령대','연락달', '혜택분야',  '연회비납부정도',
#        '대학교구분', '통화시간_분', '직업급여별',  '통화시간(분)', '소개']]

In [ ]:
# temp2.loc[:,'연령대'] = temp2['연령'].apply(age_group)

In [ ]:
# temp2.to_csv('텔레마케팅_모델링2.csv')

In [ ]:
df['성공여부'] = np.where(df['계약여부'] == 'yes', 1, 0)

In [ ]:
df['통화시간_분'].unique()

In [ ]:
df[df['통화시간_분'] == min][['통화시간_분','성공여부']].mean()

In [ ]:
print(len(df['통화시간'].unique()))
len(df['통화시간'])

# 연락달별 성공률

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
month = df.groupby('연락달',as_index = False)['성공여부'].agg('mean')
plt.bar(month['연락달'], month['성공여부'])
plt.ylim((0.08,0.16))

# 통화시간별 계약성공률

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
df['통화시간(분)'] = df['통화시간_분'].astype(int)

In [ ]:
time = df.groupby('통화시간_분', as_index = False)['성공여부'].agg('mean').round(3)
cond = time['통화시간_분'] < 30
temp = time[cond]
plt.scatter(temp['통화시간_분'],temp['성공여부'])

## 통화성공률이 떨어지는 경우 제거 

In [ ]:
idx = []
for i in range(len(time)-1):
    if (time['성공여부'][i] - time['성공여부'][i+1]) > 0:
        idx.append(i)

In [ ]:
temp = time.copy()


In [ ]:
tmp = temp.iloc[[i for i in temp.index if i not in idx]]
tmp = tmp.iloc[:27]
plt.scatter(tmp['통화시간(분)'],tmp['성공여부'])

### 통화시간별 성공률

In [ ]:
tmp.to_csv('telemarketing_time.csv')

# 과거 통화횟수별 성공율

In [ ]:
count = []; count_ratio = [];
for i in df['과거통화횟수'].unique():
    count.append(i)
    count_ratio.append(df[df['과거통화횟수'] == i]['성공여부'].mean())

In [ ]:
pd.DataFrame({'횟수':count,'횟수별비율':count_ratio})

In [ ]:
plt.bar(count, count_ratio)

# p_days별 성공횟수

In [ ]:
tmp = df[df['성공여부'] == 1].groupby('p_days',as_index = False)['성공여부'].agg('count').rename(columns = {'성공여부':'성공횟수'})
total_success = df.groupby('p_days',as_index = False)['성공여부'].agg('count').rename(columns = {'성공횟수':'총성공횟수'})

In [ ]:
p_days  = pd.merge( tmp,total_success,on='p_days', how = 'left')
i = 1
p_ch = pd.DataFrame()
for _ in range(1,4):
    if _ == 1:
        p_ch[f'{i}에서{i+9}'] = p_days.iloc[i:i+9].sum()
        i = 8 +i
    else:
        p_ch[f'{i}에서{i+8}'] = p_days.iloc[i:i+9].sum()
        i = 8 +i
p_ch = p_ch.T
p_ch = p_ch.reset_index().iloc[:,1:]
p_ch['p_days별 성공률'] = (p_ch['성공횟수']/p_ch['성공여부']).round(3)
p_ch.columns = ['8묶음씩 p_days','p_days성공횟수', 'p_days성공여부', 'p_days별 성공률']
p_ch

In [ ]:
1195.0	+ 311.0 + 37

In [ ]:
plt.bar(p_ch['8묶음씩 p_days'], p_ch['p_days별 성공률'])

In [ ]:
contact = []; contact_ratio = [];
for i in df['연락시도횟수'].unique():
    contact.append(i)
    contact_ratio.append(df[df['연락시도횟수'] == i]['성공여부'].mean())

In [ ]:
plt.scatter(contact, contact_ratio)

In [ ]:
df.columns

In [ ]:
df.columns

In [ ]:
df = pd.merge(df,card[['Card_ID','소개']],left_on = '카드종류', right_on = 'Card_ID', how = 'left')
del df['Card_ID']

In [ ]:
# 모델에 들어갈 피처 
# df.drop(['연령','직업군','신용카드소지여부','contact','연락날','연락요일','공휴일과평일','혜택분야','연회비','연회비납부정도','대학교구분','통화시간_분'],axis = 1)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib
matplotlib.rc('font',family = 'NanumGothic')

In [ ]:
def count_ratio(df,col):
    ratio= df.groupby(col, as_index = False)['성공여부'].agg('mean').round(3).rename(columns = {'성공여부':'비율'})
    plt.bar(ratio[col],ratio['비율'])

In [ ]:
count_ratio(df,'대학교구분')

In [ ]:
col = '연락달'
df.groupby(col, as_index = False)['성공여부'].agg('mean').round(3)

In [ ]:

import matplotlib

In [ ]:
matplotlib.rc('font',family = 'NanumGothic')

In [ ]:
def conti_ratio(df,col):
    ratio= df.groupby(col, as_index = False)['성공여부'].agg('mean').round(3).rename(columns = {'성공여부':'비율'})
    plt.scatter(ratio[col],ratio['비율'])
    return ratio
contact_ratio = conti_ratio(df,'연락시도횟수')

In [ ]:
contact_ratio.iloc[:20].to_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/시각화/연락시도횟수별성공률.csv')

In [ ]:
df.columns

In [ ]:
df_sav = df[['연령', '직업군', '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류', '연락일', '연령대',
       '연락년도', '연락달', '연락날', '연락요일', '공휴일과평일', '혜택분야', '연회비', 
       '대학교구분',  '직업급여별', '통화시간(분)', '연령대_재범주화']]

In [ ]:
df_sav.to_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/ipynb파일/모델링_csv파일/model3.csv')

In [ ]:
df_sav.to_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/ipynb파일/모델링_csv파일/model2.csv')

In [ ]:
df_sav2 = df[['연령',  '결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류',
        '연락달', '연락날', '연락요일', '공휴일과평일', '혜택분야', '연회비', '연회비납부정도',
       '대학교구분',  '직업급여별', '성공여부', '통화시간(분)', '연령대_재범주화']]

In [ ]:
df_sav2.to_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/ipynb파일/모델링_csv파일/model1.csv')

In [ ]:
df_sav3 = df[['연령','결혼여부', '학력', '신용카드소지여부', '주택대출여부', '개인대출여부', 'contact',
       '통화시간', '연락시도횟수', 'p_days', '과거통화횟수', '계약여부', '카드종류', '연락달', '연락날', '혜택분야', '연회비', '연회비납부정도',
       '대학교구분', '직업급여별', '성공여부', '통화시간(분)', '연령대_재범주화']]

In [ ]:
df_sav3.to_csv('/home/piai/바탕화면/파이썬_기초/빅데이터프로젝트/ipynb파일/모델링_csv파일/model3.csv')